# Escola de Dados - Python para inovação cívica

## Módulo 2: Análise exporatória de dados do Legislativo

* [Aula 02: Acessando os dados da CEAP via API](#Aula-02:-Acessando-os-dados-da-CEAP-via-API)
* [Aula 03: Lendo os dados com Pandas](#Aula-03:-Lendo-os-dados-com-Pandas)
    * [Dicionário de dados](#Dicionário-de-Dados)
* [Aula 04: Organizando e visualizando os dados](#Aula-04:-Organizando-e-visualizando-os-dados)
    * [Preenchendo valores nulos](#Preenchendo-valores-nulos)
    * [Informando tipos na leitura dos dados](#Informando-tipos-na-leitura-dos-dados)
    * [as type](#astype)
    * [to_datetime](#to_datetime)
    * [completando, substituindo e reformatando](#completando,-substituindo-e-reformatando)
* [Aula 07: Dados da Receita Federal](#Aula-07:-Dados-da-Receita-Federal)
* [Aula 08: Receita Federal e CEAP](#Aula-8:-Receita-Federal-e-CEAP)

### Aula 02: Acessando os dados da CEAP via API

Para baixar os arquivos das despesas pela Cota para Exercício da Atividade Parlamentar, podemos utilizar a API da Câmara Legislativa. Os arquivos de cada ano estão disponíveis via URLs no formato `http://www.camara.leg.br/cotas/Ano-{ano}.{formato}[.zip]`.

Você pode encontrar mais detalhes na [documentação dos dados sobre Despesas pela Cota para Exercício da Atividade Parlamentar](https://dadosabertos.camara.leg.br/swagger/api.html#staticfile).

Há diversas formas de fazer o download dos arquivos usando Python. Para esse tutorial, vamos usar o pacote `requests`.

In [1]:
url = f"http://www.camara.leg.br/cotas/Ano-2019.csv.zip"

In [2]:
import requests

# fazendo o download do conteúdo do arquivo
r = requests.get(url)

In [3]:
# abrindo um arquivo no seu computador
file = open(f"Ano-2019.csv.zip", "wb")

# escrever o conteúdo
file.write(r.content)
file.close()

In [4]:
# outra forma de criar arquivos, e escrever conteúdo

# with open(f"Ano-{year}.csv.zip", "wb") as code:
#    code.write(r.content)

Como o arquivo baixado pela API é um arquivo zip, vamos precisar descompatar. Vamos usar o `zipfile`, um módulo que nos dá ferramentas para diversas manipulações de arquivos zipados.

In [5]:
from zipfile import ZipFile

In [6]:
zip_file = ZipFile(f"Ano-2019.csv.zip", 'r')

In [7]:
zip_file.namelist()

['Ano-2019.csv']

In [8]:
zip_file.extract(member=f"Ano-2019.csv", path=f"reembolso-2019")

'reembolso-2019/Ano-2019.csv'

In [ ]:
# caso a gente queira extrair todo o conteúdo do arquivo zip

# zip_file.extractall(path=f"reimbursements-{year}")

In [9]:
zip_file.close()

### Aula 03: Lendo os dados com Pandas

In [10]:
import pandas as pd

In [ ]:
pd.read_csv("reembolso-2019/Ano-2019.csv")

Opa! Erro na leitura do nosso arquivo. Quando encontramos esse erro na leitura de um arquivo, existe algumas possíveis causas mais comuns, uma delas é que o caracter que separa as colunas pode não ser o que o `read_csv` espera.

Encontre na [documentação do read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) qual o separador considerado padrão e depois confira no arquivo qual o separador que é utilizado.

Agora, modifique a próxima célula com o que você acha que deve ser feito para que a leitura do csv aconteça sem erros.

In [13]:
df_reimbursements = pd.read_csv("reembolso-2019/Ano-2019.csv", sep=";")
# sep == delimiter

Vamos visualizar as 5 primeiras linhas do nosso Dataframe.

In [ ]:
df_reimbursements.head()

## Como podemos fazer para visualizar as 10 primeiras linhas?

Escreva e rode o código na célula abaixo.

In [ ]:
df_reimbursements.head(10)

Podemos também recuperar uma amosta aleatória do nosso Dataframe. Para isso, usamos o [`sample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [ ]:
df_reimbursements.sample()

## Como podemos fazer para visualizar 5 amostras aleatórias?

In [ ]:
df_reimbursements.sample(5)

#### Dicionário de Dados

Um passo importante para dar antes de começar a análisar é entender os dados que estamos manipulando.

*Sempre procure o significado dos campos, procure entender quais os tipos de dados preenchem aquele campo. São dados numéricos? São caracteres? Não palavras usadas para categorizar/classificar os dados?*

| Elemento de Dado          | Nome do Dado                           | Definição do Dado                                                                                                                                                                                                                                                                                                                                                                                                                   |   |   |
|---------------------------|----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---|---|
| txNomeParlamentar         | Nome Parlamentar                       | Nome adotado pelo Parlamentar ao tomar posse do seu mandato. Compõe-se de dois elementos: um prenome e o nome; dois nomes; ou dois prenomes, salvo, a juízo do Presidente da Casa legislativa, que poderá alterar essa regra para que não ocorram confusões.                                                                                                                                                                        |   |   |
| ideCadastro               | Identificador Único do Parlamentar     | Número que identifica unicamente um deputado federal na CD.                                                                                                                                                                                                                                                                                                                                                                         |   |   |
| nuCarteiraParlamentar     | Número da  Carteira Parlamentar        | Documento usado para identificar um deputado federal na CD. Pode alterar a cada Legislatura nova.                                                                                                                                                                                                                                                                                                                                   |   |   |
| nuLegislatura             | Número da  Legislatura                 | No contexto da cota CEAP, representa o ano base de início da legislatura e é utilizado para compor a Carteira Parlamentar, pois esta poderá ser alterada à medida que se muda de Legislatura.                                                                                                                                     |   |   |
| sgUF                      | Sigla da UF                            | No contexto da cota CEAP, representa a unidade da federação pela qual o deputado foi eleito e é utilizada para definir o valor da cota a que o deputado tem.                                                                                                                                                                                                                                                                        |   |   |
| sgPartido                 | Sigla do Partido                       | O seu conteúdo representa a sigla de um partido. Tem personalidade jurídica de direito privado e goza de autonomia e liberdade no que diz respeito à criação, organização e funcionamento, observados os princípios e preceitos constitucionais. |   |   |
| codLegislatura            | Código da Legislatura                  | No contexto da cota CEAP, o seu conteúdo representa o código identificador da Legislatura, que um número ordinal sequencial, alterado de um em um, a cada início de uma nova Legislatura (por exemplo, a Legislatura que iniciou em 2011 é a 54ª Legislatura).                                                                    |   |   |
| numSubCota                | Número da Subcota                      | No contexto da Cota CEAP, o conteúdo deste dado representa o código do Tipo de Despesa referente à despesa realizada pelo deputado e comprovada por meio da emissão de um documento fiscal, a qual é debitada na cota do deputado.                                                                                                                                                                                                  |   |   |
| txtDescricao              | Descrição da Subcota                   | O seu conteúdo é a descrição do Tipo de Despesa relativo à despesa em questão.                                                                                                                                                                                                                                                                                                                                                      |   |   |
| numEspecificacaoSubCota   | Número da Especificação da Subcota     | No contexto da Cota CEAP, há despesas cujo Tipo de Despesa necessita ter uma especificação mais detalhada (por exemplo, “Combustível”). O conteúdo deste dado representa o código desta especificação mais detalhada.                                                                                                                                                                                                               |   |   |
| txtDescricaoEspecificacao | Descrição da Especificação da Subcota  | Representa a descrição  especificação mais detalhada de um referido Tipo de Despesa.                                                                                                                                                                                                                                                                                                                                                |   |   |
| txtFornecedor             | Fornecedor                             | O conteúdo deste dado representa o nome do fornecedor do produto ou serviço presente no documento fiscal                                                                                                                                                                                                                                                                                                                            |   |   |
| txtCNPJCPF                | CNPJ/CPF                               | O conteúdo deste dado representa o CNPJ ou o CPF do emitente do documento fiscal, quando se tratar do uso da cota em razão do reembolso despesas comprovadas pela emissão de documentos fiscais.                                                                                                                                                                                                                                    |   |   |
| txtNumero                 | Número do Documento                    | O conteúdo deste dado representa o número de face do documento fiscal emitido ou o número do documento que deu causa à despesa debitada na cota do deputado.                                                                                                                                                                                                                                                                        |   |   |
| indTipoDocumento          | Indicativo de Tipo de Documento Fiscal | Este dado representa o tipo de documento do fiscal – 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.                                                                                                                                                                                                                                                                                                |   |   |
| datEmissao                | Data de Emissão                        | O conteúdo deste dado é a data de emissão do documento fiscal ou a data do documento que tenha dado causa à despesa.                                                                                                                                                                                                                                                                                                                |   |   |
| vlrDocumento              | Valor do Documento                     | O seu conteúdo é o valor de face do documento fiscal ou o valor do documento que deu causa à despesa. Quando se tratar de bilhete aéreo, esse valor poderá ser negativo, significando que o referido bilhete é um bilhete de compensação, pois compensa um outro bilhete emitido e não utilizado pelo deputado (idem para o dado vlrLiquido abaixo).                                                                                |   |   |
| vlrGlosa                  | Valor da Glosa                         | O seu conteúdo representa o valor da glosa do documento fiscal que incidirá sobre o Valor do Documento, ou o valor da glosa do documento que deu causa à despesa.                                                                                                                                                                                                                                                                   |   |   |
| vlrLiquido                | Valor Líquido                          | O seu conteúdo representa o valor líquido do documento fiscal ou do documento que deu causa à despesa e será calculado pela diferença entre o Valor do Documento e o Valor da Glosa. É este valor que será debitado da cota do deputado. Caso o débito seja do Tipo Telefonia e o valor seja igual a zero, significa que a despesa foi franqueada.                                                                                  |   |   |
| numMes                    | Mês                                    | O seu conteúdo representa o Mês da competência financeira do documento fiscal ou do documento que deu causa à despesa. É utilizado, junto com o ano, para determinar em que período o débito gerará efeito financeiro sobre a cota.                                                                                                                                                                                                 |   |   |
| numAno                    | Ano                                    | O seu conteúdo representa o Ano da competência financeira do documento fiscal ou do documento que deu causa à despesa. É utilizado, junto com o mês, para determinar em que período o débito gerará efeito financeiro sobre a cota.                                                                                                                                                                                                 |   |   |
| numParcela                | Número da Parcela                      | O seu conteúdo representa o número da parcela do documento fiscal. Ocorre quando o documento tem de ser reembolsado de forma parcelada.                                                                                                                                                                                                                                                                                             |   |   |
| txtPassageiro             | Passageiro                             | O conteúdo deste dado representa o nome do passageiro, quando o documento que deu causa à despesa se tratar de emissão de bilhete aéreo.                                                                                                                                                                                                                                                                                            |   |   |
| txtTrecho                 | Trecho                                 | O conteúdo deste dado representa o trecho da viagem, quando o documento que deu causa à despesa se tratar de emissão de bilhete aéreo.                                                                                                                                                                                                                                                                                              |   |   |
| numLote                   | Número do Lote                         | No contexto da Cota CEAP, o Número do Lote representa uma capa de lote que agrupa os documentos que serão entregues à Câmara para serem ressarcidos. Este dado, juntamente com o Número do Ressarcimento, auxilia a localização do documento no Arquivo da Casa.                                                                                                                                                                    |   |   |
| numRessarcimento          | Número do Ressarcimento                | No contexto da Cota CEAP, o Número do Ressarcimento indica o ressarcimento do qual o documento fez parte por ocasião do processamento do seu reembolso. Este dado, juntamente com o Valor da Restituição, auxilia a localização do documento no Arquivo da Casa.                                                                                                                                                                 |   |   |
|  vlrRestituicao           |  Valor da Restituição                  |  O seu conteúdo representa o valor restituído do documento fiscal que incidirá sobre o Valor do Documento.                                                                                                                                                                                                                                                                                                                          |   |   |
| nuDeputadoId              | Identificador do Solicitante           | Número que identifica um Parlamentar ou Liderança na Transparência da Cota para Exercício da Atividade Parlamentar.                                                                                                                                                                                                                                                                                                                 |   |   |


Legislatura: Período de quatro anos coincidente com o mandato parlamentar dos Deputados Federais.

Definição de partido: é uma organização formada por pessoas com interesse ou ideologia comuns, que se associam com o fim de assumir o poder para implantar um programa de governo.

*Fonte*: [EXPLICAÇÕES SOBRE O FORMATO DOS ARQUIVOS XML](https://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/explicacoes-sobre-o-formato-dos-arquivos-xml)

### Aula 04: Organizando e visualizando os dados

Vamos conhecer mais algumas operações que podem nos ajudar a visualizar e entender nossos dados.

In [40]:
DTYPE = {
    "txNomeParlamentar": str,
    "ideCadastro": str,
    "nuCarteiraParlamentar": str,
    "nuLegislatura": str,
    "sgUF": str,
    "sgPartido": str,
    "codLegislatura": str,
    "numSubCota": str,
    "txtDescricao": str,
    "numEspecificacaoSubCota": str,
    "txtDescricaoEspecificacao": str,
    "txtFornecedor": str,
    "txtCNPJCPF": str,
    "txtNumero": str,
    "indTipoDocumento": str,
    "datEmissao": str,
    "vlrDocumento": float,
    "vlrGlosa": str,
    "vlrLiquido": float,
    "numMes": str,
    "numAno": str,
    "numParcela": str,
    "txtPassageiro": str,
    "txtTrecho": str,
    "numLote": str,
    "numRessarcimento": str,
    "nuDeputadoId": str,
    "ideDocumento": str,
}

df_reembolso = pd.read_csv(
    "reembolso-2019/Ano-2019.csv", delimiter=";", dtype=DTYPE, low_memory=False
)

In [ ]:
df_reimbursements.tail()

In [ ]:
df_reimbursements.shape

(289453, 32)

In [18]:
df_reimbursements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289453 entries, 0 to 289452
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          289453 non-null  object 
 1   cpf                        288449 non-null  float64
 2   ideCadastro                288548 non-null  float64
 3   nuCarteiraParlamentar      288548 non-null  float64
 4   nuLegislatura              289453 non-null  int64  
 5   sgUF                       288548 non-null  object 
 6   sgPartido                  288548 non-null  object 
 7   codLegislatura             289453 non-null  int64  
 8   numSubCota                 289453 non-null  int64  
 9   txtDescricao               289453 non-null  object 
 10  numEspecificacaoSubCota    289453 non-null  int64  
 11  txtDescricaoEspecificacao  64765 non-null   object 
 12  txtFornecedor              289438 non-null  object 
 13  txtCNPJCPF                 28

astype

In [ ]:

# df_reembolso["ideCadastro"] = df_reembolso["ideCadastro"].astype('string')

In [19]:
df_reimbursements.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'datPagamentoRestituicao', 'vlrRestituicao',
       'nuDeputadoId', 'ideDocumento', 'urlDocumento'],
      dtype='object')

#### Preenchendo valores nulos

In [ ]:
# TESTE MEU : PREENCHER APENAS CAMPOS NÚLOS DE UMA DETERMINADA COLUNA
df_reembolso["vlrRestituicao"] = df_reembolso["vlrRestituicao"].fillna("Indisponível")
df_reembolso.head()

In [ ]:
df_reimbursements.fillna(value="Não se aplica").head()

In [ ]:
# Caso deseje aplicar a sugestão de transformação, execute:
# df_reembolso = df_reembolso.fillna(value="Não se aplica")

In [44]:
df_reimbursements[df_reimbursements.sgPartido.isna()]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LID.GOV-CD,NaN,NaN,NaN,2015,NaN,NaN,55,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1554246,NaN,NaN,NaN,2812,6738121,https://www.camara.leg.br/cota-parlamentar/not...
1,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1570078,NaN,NaN,NaN,2812,6769606,https://www.camara.leg.br/cota-parlamentar/not...
2,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1570503,NaN,NaN,NaN,2812,6771099,https://www.camara.leg.br/cota-parlamentar/not...
3,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1579375,NaN,NaN,NaN,2812,6793690,https://www.camara.leg.br/cota-parlamentar/not...
4,LID.GOV-CD,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1590980,NaN,NaN,NaN,2812,6821674,https://www.camara.leg.br/cota-parlamentar/not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,LIDMIN,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1618189,NaN,NaN,NaN,2806,6884032,https://www.camara.leg.br/cota-parlamentar/not...
901,LIDMIN,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1658394,NaN,NaN,NaN,2806,6976028,https://www.camara.leg.br/cota-parlamentar/doc...
902,LIDMIN,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1658383,NaN,NaN,NaN,2806,6975999,https://www.camara.leg.br/cota-parlamentar/doc...
903,LIDMIN,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,0,NaN,NaN,1658375,NaN,NaN,NaN,2806,6975967,https://www.camara.leg.br/cota-parlamentar/doc...


In [45]:
df_reimbursements[df_reimbursements['ideDocumento'].isnull()]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento


#### Informando tipos na leitura dos dados

Quando trabalhamos com pandas, é sempre interessante entender que tipos de dados estão contidos em cada coluna. Na leitura do arquivo podemos aproveitar para deixar isso configurado.

Isso pode ser extramamente útil quando estamos lendo dados que contém informação sobre documentos, como CPF. Nesses casos podemos ter algo como `091777777`, se esse valor por lido como um número, ele vai virar `91777777`, já que não existe número que inicie com 0. Esse é um ótimo exemplo de dado que deve ser lido como um string, para que ele seja preservado ao máximo.

Agora, sabendo quais dados nosso dataframe possui, vamos definir o tipo para cada um.

In [46]:
DTYPE = {
    'txNomeParlamentar': str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str,
}

df_reimbursements = pd.read_csv(
    "reembolso-2019/Ano-2019.csv",
    delimiter=";",
    dtype=DTYPE,
    low_memory=False
)

Opa! Você lembra qual método podemos usar para conferir os tipos de cada coluna?
**dica**: já passamos por esse método...

In [47]:
df_reimbursements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289453 entries, 0 to 289452
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   txNomeParlamentar          289453 non-null  object 
 1   cpf                        288449 non-null  float64
 2   ideCadastro                288548 non-null  object 
 3   nuCarteiraParlamentar      288548 non-null  object 
 4   nuLegislatura              289453 non-null  object 
 5   sgUF                       288548 non-null  object 
 6   sgPartido                  288548 non-null  object 
 7   codLegislatura             289453 non-null  object 
 8   numSubCota                 289453 non-null  object 
 9   txtDescricao               289453 non-null  object 
 10  numEspecificacaoSubCota    289453 non-null  object 
 11  txtDescricaoEspecificacao  64765 non-null   object 
 12  txtFornecedor              289438 non-null  object 
 13  txtCNPJCPF                 28

#### astype

In [ ]:
# df_reimbursements["ideCadastro"] = df_reimbursements["ideCadastro"].astype('string')

#### to_datetime

Por default, o `to_datetime()` vai converter string no formato `MM/DD`, `MMDD` ou `MM-DD`. No nosso caso, as datas estão em um formato diferente, por isso, vamos passar como argumento do `format`o valor `%Y-%m-%d`, informando que as datas possuem primeiro o *ano* completo (ex.: 2020), *mês (ex.: 05), e dia (ex.: 12).

In [48]:
df_reimbursements["datEmissao"] = pd.to_datetime(
    df_reimbursements.datEmissao,
    format='%Y-%m-%d'
)

Caso essa conversão gere algum erro, você pode informar ao pandas como ele deve lidar com isso. O `to_datetime()` possue o argumento `errors`, que pode receber `ignore` caso você queria que os erros sejam ignorados, ou `coerce` caso você queria que os valores inválidos recebam `NaT`.

##### Para quem quiser saber mais!

Você deve estar se perguntando o porquê dessa conversão utilizando o `to_datetime`. _O que ganhamos com isso?_ Existem várias vantagens do Pandas entender que uma coluna é uma data. Uma dessas vantagens é que você vai poder trabalhar com os dados como uma série temporal, ou seja, um conjunto de registros ao longo do tempo.

Os reembolsos se encaixam bem nisso... pense só, nossos dados são um conjunto de pedidos de reembolsos solicitados ao longo de um ano. Quando convertemos o dados para um datetime, podemos realizar algumas operações que só são possíveis com datas. Por exemplo, podemos selecionar uma janela de tempo:

In [26]:
df_reimbursements.set_index("datEmissao").loc["2020-01-01":"2020-04-30"]

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
datEmissao,,,,,,,,,,,,,,,,,,,,,
2020-01-10,LIDERANÇA DO DEMOCRATAS,NaN,NaN,NaN,2019,NaN,NaN,56,12,ASSINATURA DE PUBLICAÇÕES,...,0,NaN,NaN,1667101,NaN,NaN,NaN,2713,6994538,https://www.camara.leg.br/cota-parlamentar/not...
2020-01-02,LIDERANÇA DO PODEMOS,NaN,NaN,NaN,2019,NaN,NaN,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,0,NaN,NaN,1669984,NaN,NaN,NaN,3172,7000670,https://www.camara.leg.br/cota-parlamentar/not...
2020-01-04,LIDERANÇA DO PT,NaN,NaN,NaN,2019,NaN,NaN,56,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,...,0,NaN,NaN,1667165,NaN,NaN,NaN,2439,6994687,https://www.camara.leg.br/cota-parlamentar/doc...
2020-01-31,LIDERANÇA DO PT,NaN,NaN,NaN,2019,NaN,NaN,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,0,NaN,NaN,1670191,NaN,NaN,NaN,2439,7001237,https://www.camara.leg.br/cota-parlamentar/not...
2020-01-07,LIDERANÇA DO PT,NaN,NaN,NaN,2019,NaN,NaN,56,120,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,...,0,NaN,NaN,1667166,NaN,NaN,NaN,2439,6994688,https://www.camara.leg.br/cota-parlamentar/not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01,Enéias Reis,9.450581e+10,207309,533,2019,MG,UNIÃO,56,10,TELEFONIA,...,0,NaN,NaN,1669485,NaN,NaN,NaN,3446,6999178,NaN
2020-01-03,Fernando Borja,5.267385e+10,212504,547,2019,MG,AVANTE,56,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,0,NaN,NaN,1675071,NaN,NaN,NaN,3462,7012146,https://www.camara.leg.br/cota-parlamentar/doc...
2020-02-28,Fernando Borja,5.267385e+10,212504,547,2019,MG,AVANTE,56,10,TELEFONIA,...,0,NaN,NaN,0,0,NaN,NaN,3462,0,NaN


Outras coisas são possíveis, como encontrar o dia da semana de cada uma dessas datas, calcular a média móvel, calcular quantos anos se passaram, ou agrupar os dados por janelas de tempo.

#### Completando, substituindo e reformatando

In [27]:
# vamos trocar qualquer caracter que não seja um dígito número por '', essa operação remove os dígitos
df_reimbursements["txtCNPJCPF"] = df_reimbursements["txtCNPJCPF"].str.replace(r'\D', '', regex=True)

In [28]:
# Este dado representa o tipo de documento do fiscal –
# 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.

df_reimbursements.indTipoDocumento.unique()

array(['4', '0', '1', '3', '2'], dtype=object)

In [29]:
converters = {
    "0": 'nota_fiscal',
    "1": 'recibo',
    "2": 'despesa_exterior',
    "4": None
}

converters

{'0': 'nota_fiscal', '1': 'recibo', '2': 'despesa_exterior', '4': None}

In [30]:
df_reimbursements.indTipoDocumento.replace(converters, inplace=True)

In [31]:
df_reimbursements.indTipoDocumento.unique()

array([None, 'nota_fiscal', 'recibo', '3', 'despesa_exterior'],
      dtype=object)

### Aula 07: Dados da Receita Federal

Referencias:

- http://receita.economia.gov.br/sobre/acordos-e-convenios/compartilhamento-de-bases-de-dados-2013-decreto-no-8-789-2016/leiaute-das-bases/dados-da-base-cnpj
- http://200.152.38.155/CNPJ/LAYOUT_DADOS_ABERTOS_CNPJ.pdf


situation (Situação cadastral)

    1 – NULA: o CNPJ torna-se inválido, seja por inconformidade de dados, fraudes ou outros motivos.
    2 – ATIVA: significa que a empresa está em dia com todas as suas pendências, não há nenhuma irregularidade e os pagamentos estão em dia;
    3 – SUSPENSA: a empresa tem pendências com a Receita Federal, não cumpriu com suas obrigações ou pagamentos ou está sendo investigada por possível fraude;
    4 – INAPTA: a empresa está a dois anos ou mais com suspensão e não apresentou os devidos dados contábeis;
    8 – BAIXADA: a empresa foi extinta, seja por solicitação dos sócios ou por ação da Receita;
    
    
situation_date (Data da situação cadastral): DATA DO EVENTO DA SITUACAO CADASTRAL

In [32]:
import requests

url = "https://serenata-de-amor-data.nyc3.digitaloceanspaces.com/2019-11-19-companies.csv.xz"

# fazendo o download do conteúdo do arquivo
r = requests.get(url)

# abrindo um arquivo no seu computador
file = open("2019-11-19-companies.csv.xz", "wb")

# escrever o conteúdo
file.write(r.content)
file.close()

In [33]:
DTYPE = {
    'cnpj': str,
}

df_empresas = pd.read_csv(
    '2019-11-19-companies.csv.xz', # .xz é um arquivo compactado
    compression='xz',
    dtype=DTYPE,
)
df_empresas.head(1)

,cnpj,partners,secondary_activities,latitude,longitude,opening,main_activity,neighborhood,share_capital,zip_code,...,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
0,08771105000156,"[{""id"": 2, ""name"": ""MARLY MORAIS DA GAMA"", ""cn...",NaN,NaN,NaN,NaN,NaN,REALEZA,10000.0,36900000.0,...,0,1,49,WERNER COMERCIAL DE ALIMENTOS LTDA,2,NaN,NaN,NaN,MG,NaN


In [34]:
df_empresas['situation_date'] = pd.to_datetime(
    df_empresas['situation_date'],
    format="%Y-%m-%d"
)

In [35]:
df_empresas["cnpj"] = df_empresas["cnpj"].str.replace(r'\D', '', regex=True)

### Aula 8: Receita Federal e CEAP

In [36]:
df_final = df_reimbursements.merge(
    df_empresas,
    how='left',
    left_on='txtCNPJCPF',
    right_on='cnpj'
)

### Exportando o resultado

In [37]:
df_reimbursements.to_csv('reembolso-2019-completo.csv')